# Exploratory Data Analysis - Food Delivery Times

This notebook explores the food delivery dataset to understand patterns, outliers, and relationships that will inform our modeling approach.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Data Loading and Overview

In [ ]:
# Load the dataset
df = pd.read_csv('../data/Food_Delivery_Times.csv')

print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nFirst few rows:")
df.head()

In [ ]:
# Basic info about the dataset
print("Data Types:")
print(df.dtypes)
print("\nMissing Values:")
print(df.isnull().sum())
print("\nBasic Statistics:")
df.describe()

## 2. Target Variable Analysis

In [ ]:
# Distribution of delivery times
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram
axes[0].hist(df['Delivery_Time_min'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_title('Distribution of Delivery Times')
axes[0].set_xlabel('Delivery Time (minutes)')
axes[0].set_ylabel('Frequency')

# Box plot
axes[1].boxplot(df['Delivery_Time_min'])
axes[1].set_title('Box Plot of Delivery Times')
axes[1].set_ylabel('Delivery Time (minutes)')

# Q-Q plot
from scipy import stats
stats.probplot(df['Delivery_Time_min'], dist="norm", plot=axes[2])
axes[2].set_title('Q-Q Plot of Delivery Times')

plt.tight_layout()
plt.show()

print(f"Mean delivery time: {df['Delivery_Time_min'].mean():.2f} minutes")
print(f"Median delivery time: {df['Delivery_Time_min'].median():.2f} minutes")
print(f"Standard deviation: {df['Delivery_Time_min'].std():.2f} minutes")
print(f"Min delivery time: {df['Delivery_Time_min'].min():.2f} minutes")
print(f"Max delivery time: {df['Delivery_Time_min'].max():.2f} minutes")

## 3. Feature Analysis

In [ ]:
# Categorical features analysis
categorical_features = ['Weather', 'Traffic_Level', 'Time_of_Day', 'Vehicle_Type']

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for i, feature in enumerate(categorical_features):
    if feature in df.columns:
        # Value counts
        counts = df[feature].value_counts()
        axes[i].bar(counts.index, counts.values, alpha=0.7)
        axes[i].set_title(f'Distribution of {feature}')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('Count')
        axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Numerical features analysis
numerical_features = ['Distance_km', 'Preparation_Time_min', 'Courier_Experience_yrs']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, feature in enumerate(numerical_features):
    if feature in df.columns:
        axes[i].hist(df[feature], bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
        axes[i].set_title(f'Distribution of {feature}')
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 4. Correlation Analysis

In [ ]:
# Correlation matrix for numerical features
numerical_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numerical_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Matrix of Numerical Features')
plt.tight_layout()
plt.show()

# Strongest correlations with delivery time
delivery_corr = correlation_matrix['Delivery_Time_min'].sort_values(key=abs, ascending=False)
print("Correlations with Delivery Time:")
print(delivery_corr)

## 5. Feature Relationships with Target

In [ ]:
# Weather vs Delivery Time
if 'Weather' in df.columns:
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    sns.boxplot(data=df, x='Weather', y='Delivery_Time_min')
    plt.title('Delivery Time by Weather Condition')
    plt.xticks(rotation=45)
    
    plt.subplot(1, 2, 2)
    weather_stats = df.groupby('Weather')['Delivery_Time_min'].agg(['mean', 'std', 'count'])
    weather_stats['mean'].plot(kind='bar', alpha=0.7, color='lightblue')
    plt.title('Average Delivery Time by Weather')
    plt.ylabel('Average Delivery Time (min)')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    print("Weather Impact on Delivery Time:")
    print(weather_stats)

In [ ]:
# Traffic Level vs Delivery Time
if 'Traffic_Level' in df.columns:
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    sns.boxplot(data=df, x='Traffic_Level', y='Delivery_Time_min')
    plt.title('Delivery Time by Traffic Level')
    
    plt.subplot(1, 2, 2)
    traffic_stats = df.groupby('Traffic_Level')['Delivery_Time_min'].agg(['mean', 'std', 'count'])
    traffic_stats['mean'].plot(kind='bar', alpha=0.7, color='orange')
    plt.title('Average Delivery Time by Traffic Level')
    plt.ylabel('Average Delivery Time (min)')
    
    plt.tight_layout()
    plt.show()
    
    print("Traffic Impact on Delivery Time:")
    print(traffic_stats)

In [ ]:
# Distance vs Delivery Time scatter plot
if 'Distance_km' in df.columns:
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.scatter(df['Distance_km'], df['Delivery_Time_min'], alpha=0.6, color='green')
    plt.xlabel('Distance (km)')
    plt.ylabel('Delivery Time (min)')
    plt.title('Distance vs Delivery Time')
    
    # Add trend line
    z = np.polyfit(df['Distance_km'], df['Delivery_Time_min'], 1)
    p = np.poly1d(z)
    plt.plot(df['Distance_km'], p(df['Distance_km']), "r--", alpha=0.8)
    
    plt.subplot(1, 2, 2)
    # Distance bins analysis
    df['Distance_Bins'] = pd.cut(df['Distance_km'], bins=5, labels=['Very Short', 'Short', 'Medium', 'Long', 'Very Long'])
    distance_stats = df.groupby('Distance_Bins')['Delivery_Time_min'].mean()
    distance_stats.plot(kind='bar', alpha=0.7, color='purple')
    plt.title('Average Delivery Time by Distance Range')
    plt.ylabel('Average Delivery Time (min)')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

## 6. Outlier Detection

In [ ]:
# Outlier detection using IQR method
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Check outliers in key numerical columns
outlier_features = ['Delivery_Time_min', 'Distance_km', 'Preparation_Time_min']

for feature in outlier_features:
    if feature in df.columns:
        outliers, lower, upper = detect_outliers_iqr(df, feature)
        print(f"\n{feature} Outliers:")
        print(f"Number of outliers: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")
        print(f"Bounds: [{lower:.2f}, {upper:.2f}]")
        
        if len(outliers) > 0:
            print(f"Outlier range: {outliers[feature].min():.2f} - {outliers[feature].max():.2f}")

## 7. Time and Vehicle Analysis

In [ ]:
# Time of day analysis
if 'Time_of_Day' in df.columns:
    time_analysis = df.groupby('Time_of_Day')['Delivery_Time_min'].agg(['mean', 'std', 'count'])
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    time_analysis['mean'].plot(kind='bar', alpha=0.7, color='teal')
    plt.title('Average Delivery Time by Time of Day')
    plt.ylabel('Average Delivery Time (min)')
    plt.xticks(rotation=45)
    
    plt.subplot(1, 2, 2)
    sns.boxplot(data=df, x='Time_of_Day', y='Delivery_Time_min')
    plt.title('Delivery Time Distribution by Time of Day')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    print("Time of Day Analysis:")
    print(time_analysis)

In [ ]:
# Vehicle type analysis
if 'Vehicle_Type' in df.columns:
    vehicle_analysis = df.groupby('Vehicle_Type')['Delivery_Time_min'].agg(['mean', 'std', 'count'])
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    vehicle_analysis['mean'].plot(kind='bar', alpha=0.7, color='coral')
    plt.title('Average Delivery Time by Vehicle Type')
    plt.ylabel('Average Delivery Time (min)')
    
    plt.subplot(1, 2, 2)
    sns.boxplot(data=df, x='Vehicle_Type', y='Delivery_Time_min')
    plt.title('Delivery Time Distribution by Vehicle Type')
    
    plt.tight_layout()
    plt.show()
    
    print("Vehicle Type Analysis:")
    print(vehicle_analysis)

## 8. Multi-dimensional Analysis

In [ ]:
# Weather + Traffic interaction
if 'Weather' in df.columns and 'Traffic_Level' in df.columns:
    plt.figure(figsize=(12, 8))
    
    # Pivot table for heatmap
    weather_traffic = df.pivot_table(values='Delivery_Time_min', 
                                   index='Weather', 
                                   columns='Traffic_Level', 
                                   aggfunc='mean')
    
    sns.heatmap(weather_traffic, annot=True, cmap='YlOrRd', fmt='.1f')
    plt.title('Average Delivery Time by Weather and Traffic Level')
    plt.tight_layout()
    plt.show()
    
    print("Weather + Traffic Interaction:")
    print(weather_traffic)

## 9. Feature Engineering Insights

In [ ]:
# Create potential new features for analysis
df_analysis = df.copy()

# Speed calculation (if distance and time are available)
if 'Distance_km' in df.columns:
    df_analysis['Speed_kmh'] = (df_analysis['Distance_km'] / df_analysis['Delivery_Time_min']) * 60
    
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.hist(df_analysis['Speed_kmh'], bins=30, alpha=0.7, color='gold', edgecolor='black')
    plt.title('Distribution of Delivery Speed')
    plt.xlabel('Speed (km/h)')
    plt.ylabel('Frequency')
    
    plt.subplot(1, 2, 2)
    if 'Vehicle_Type' in df.columns:
        sns.boxplot(data=df_analysis, x='Vehicle_Type', y='Speed_kmh')
        plt.title('Speed by Vehicle Type')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Average delivery speed: {df_analysis['Speed_kmh'].mean():.2f} km/h")
    print(f"Speed statistics:")
    print(df_analysis['Speed_kmh'].describe())

In [ ]:
# Experience impact analysis
if 'Courier_Experience_yrs' in df.columns:
    # Create experience categories
    df_analysis['Experience_Category'] = pd.cut(df_analysis['Courier_Experience_yrs'], 
                                              bins=[0, 1, 3, 5, float('inf')], 
                                              labels=['Novice', 'Beginner', 'Experienced', 'Expert'])
    
    exp_analysis = df_analysis.groupby('Experience_Category')['Delivery_Time_min'].agg(['mean', 'std', 'count'])
    
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    exp_analysis['mean'].plot(kind='bar', alpha=0.7, color='lightgreen')
    plt.title('Average Delivery Time by Experience Level')
    plt.ylabel('Average Delivery Time (min)')
    plt.xticks(rotation=45)
    
    plt.subplot(1, 2, 2)
    plt.scatter(df_analysis['Courier_Experience_yrs'], df_analysis['Delivery_Time_min'], alpha=0.6)
    plt.xlabel('Courier Experience (years)')
    plt.ylabel('Delivery Time (min)')
    plt.title('Experience vs Delivery Time')
    
    plt.tight_layout()
    plt.show()
    
    print("Experience Impact Analysis:")
    print(exp_analysis)

## 10. Key Findings Summary

In [ ]:
print("=== KEY EDA FINDINGS ===")
print()

# Statistical summary
print("1. TARGET VARIABLE (Delivery Time):")
print(f"   • Range: {df['Delivery_Time_min'].min():.1f} - {df['Delivery_Time_min'].max():.1f} minutes")
print(f"   • Mean: {df['Delivery_Time_min'].mean():.1f} ± {df['Delivery_Time_min'].std():.1f} minutes")
print(f"   • Distribution: {'Right-skewed' if df['Delivery_Time_min'].skew() > 0 else 'Left-skewed'}")
print()

# Feature importance insights
if 'Distance_km' in df.columns:
    distance_corr = df['Distance_km'].corr(df['Delivery_Time_min'])
    print(f"2. DISTANCE IMPACT:")
    print(f"   • Correlation with delivery time: {distance_corr:.3f}")
    print(f"   • Average speed: {df_analysis['Speed_kmh'].mean():.1f} km/h")
print()

# Categorical features insights
if 'Weather' in df.columns:
    weather_impact = df.groupby('Weather')['Delivery_Time_min'].mean()
    worst_weather = weather_impact.idxmax()
    best_weather = weather_impact.idxmin()
    print(f"3. WEATHER IMPACT:")
    print(f"   • Worst conditions: {worst_weather} ({weather_impact[worst_weather]:.1f} min avg)")
    print(f"   • Best conditions: {best_weather} ({weather_impact[best_weather]:.1f} min avg)")
print()

if 'Traffic_Level' in df.columns:
    traffic_impact = df.groupby('Traffic_Level')['Delivery_Time_min'].mean()
    print(f"4. TRAFFIC IMPACT:")
    for level in traffic_impact.index:
        print(f"   • {level}: {traffic_impact[level]:.1f} min average")
print()

# Data quality
missing_pct = (df.isnull().sum() / len(df) * 100)
print(f"5. DATA QUALITY:")
print(f"   • Dataset size: {len(df):,} records")
if missing_pct.sum() > 0:
    print(f"   • Missing values: {missing_pct[missing_pct > 0].to_dict()}")
else:
    print(f"   • No missing values detected")

# Outliers summary
outliers_delivery, _, _ = detect_outliers_iqr(df, 'Delivery_Time_min')
outlier_pct = len(outliers_delivery) / len(df) * 100
print(f"   • Outliers in delivery time: {outlier_pct:.1f}%")